<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/L6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import sys
nltk.download('brown')

from nltk.corpus import brown
from nltk.corpus import treebank
import nltk

# Downloading required corpus
nltk.download('punkt')
nltk.download('brown')

from nltk import word_tokenize
from nltk.corpus import brown
import pprint
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:

brown_tags_words = []
brown_tagged_sents = brown.tagged_sents()

for sent in brown_tagged_sents:
    # sent is a list of word/tag pairs
    # add START/START at the beginning
    brown_tags_words.append( ("START", "START") )
    # then all the tag/word pairs for the word/tag pairs in the sentence.
    # shorten tags to 2 characters each
    brown_tags_words.extend([ (tag[:2], word) for (word, tag) in sent ])
    # then END/END
    brown_tags_words.append( ("END", "END") )

# conditional frequency distribution
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
# conditional probability distribution
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))

# Estimating P(ti | t{i-1}) from corpus data using Maximum Likelihood Estimation (MLE):
# P(ti | t{i-1}) = count(t{i-1}, ti) / count(t{i-1})
brown_tags = [tag for (tag, word) in brown_tags_words ]

# make conditional frequency distribution:
# count(t{i-1} ti)
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# make conditional probability distribution, using
# maximum likelihood estimate:
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("DT"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))

The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05
If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'VB', the probability of 'JJ' is 0.016885067592065053
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


In [11]:
len(sentence[l])

12

In [15]:
#要的***

distinct_tags = set(brown_tags) #词性库
sentence =[
    word_tokenize("This race is awesome, I want to race too."),
    word_tokenize("That race is silly, I do not want to race.")
]
sentlen = len(sentence)
def test_model(test_samples):
  token_sequences = []
  for l in range(sentlen):
    token = []
    viterbi = [ ]
    token = []
    backpointer = [ ]

    first_viterbi = { }
    first_backpointer = { }
    for tag in distinct_tags:

        if tag == "START": continue
        first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[l][0] )
        first_backpointer[ tag ] = "START"
        
    viterbi.append(first_viterbi)
    backpointer.append(first_backpointer)

    currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])

    for wordindex in range(0, len(sentence[l])):
      this_viterbi = { }
      this_backpointer = { }
      prev_viterbi = viterbi[-1]


      for tag in distinct_tags:
          if tag == "START": continue
          for word in sentence[l]:
            best_previous = max(prev_viterbi.keys(),
                                key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[l][wordindex]))
            this_viterbi[ tag ] = prev_viterbi[ best_previous] * cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[l][wordindex])
            this_backpointer[ tag ] = best_previous
      currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
      token.append(currbest)
      viterbi.append(this_viterbi)
      backpointer.append(this_backpointer)
    token_sequences.append(token)
  return token_sequences
print(sentence)
print(test_model(sentence))

[['This', 'race', 'is', 'awesome', ',', 'I', 'want', 'to', 'race', 'too', '.'], ['That', 'race', 'is', 'silly', ',', 'I', 'do', 'not', 'want', 'to', 'race', '.']]
[['DT', 'NN', 'BE', 'JJ', ',', 'PP', 'VB', 'TO', 'NN', 'QL', '.'], ['DT', 'NN', 'BE', 'JJ', ',', 'PP', 'DO', '*', 'VB', 'TO', 'NN', '.']]
